In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/quickview of price surge in nigeria/data Vizualizations /Food CPI_Items(1995-2022)_monthly.csv')

# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df.info())


| Time       | All Items (weight = 1000.00)   | All Items less Farm Produce (weight = 513.10)   | All Items less Farm Produce and Energy (weight = 405.55)   | Imported Food (weight = 132.88)   | Food (weight = 507.06)   | Food &  Non Alcoholic Bev (weight = 518.00)   | Alcoholic Beverage, Tobacco and Kola (weight = 10.87)   | Clothing and Footwear (weight = 76.50)   | Housing Water, Electricity, Gas and Other  Fuel (weight = 167.34)   | Furnishings & Household Equipment Maintenance (weight = 50.30)   | Health (weight = 30.04)   | Transport (weight = 65.08)   | Communication (weight = 6.80)   | Recreation & Culture (weight = 6.87)   | Education (weight = 39.44)   | Restaurant & Hotels (weight = 12.12)   | Miscellaneous Goods & Services (weight = 16.63)   |
|:-----------|:-------------------------------|:------------------------------------------------|:-----------------------------------------------------------|:----------------------------------|:-------------------------|:------------

In [ ]:
import numpy as np

# Filter the dataframe to the columns: `Time`, `Food (weight = 507.06)`, `All Items (weight = 1000.00)`, `Food &  Non Alcoholic Bev (weight = 518.00)`, and `Alcoholic Beverage, Tobacco and Kola (weight = 10.87)`.
df = df[['Time', 'Food (weight = 507.06)', 'All Items (weight = 1000.00)', 'Food &  Non Alcoholic Bev (weight = 518.00)', 'Alcoholic Beverage, Tobacco and Kola (weight = 10.87)']]

# Convert the `Time` column to datetime.
df['Time'] = pd.to_datetime(df['Time'], format='%d/%m/%Y')

# Get all unique non-numeric values from all columns
for col in df.columns:
  if col != 'Time':
    # Get all unique non-numeric values excluding the first row
    non_numeric_value = df[pd.to_numeric(df[col].astype(str).str.replace('.', '', regex=False), errors='coerce').isna()][col].unique()[1:]

    if (len(non_numeric_value) > 20):
      # Sample 20 of them if there are too many unique values
      print(f"Non numeric values in column {col}: {np.random.choice(non_numeric_value, 20, replace=False)}")
    else:
      # Otherwise print all unique non-numeric values from the columns
      print(f"Non numeric values in column {col}: {non_numeric_value}")

# Drop the first row which has all missing values.
df = df.iloc[1:].copy()

# Convert the columns: `Food (weight = 507.06)`, `All Items (weight = 1000.00)`, `Food &  Non Alcoholic Bev (weight = 518.00)`, and `Alcoholic Beverage, Tobacco and Kola (weight = 10.87)` to numeric, after replacing '.' with missing values.
for col in df.columns:
  if col != 'Time':
    df[col] = pd.to_numeric(df[col].astype(str).str.replace('.', '', regex=False), errors='coerce')

# Rename the columns to: `Time`, `Food`, `All_items`, `Food_non_alcoholic_bev`, and `Alcohol_tobacco_kola` respectively.
df = df.rename(columns={'Food (weight = 507.06)': 'Food', 'All Items (weight = 1000.00)': 'All_items', 'Food &  Non Alcoholic Bev (weight = 518.00)': 'Food_non_alcoholic_bev', 'Alcoholic Beverage, Tobacco and Kola (weight = 10.87)': 'Alcohol_tobacco_kola'})

# Sort the dataframe by `Time` in ascending order.
df = df.sort_values('Time')

# Print the first 5 rows of the dataframe.
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


Non numeric values in column Food (weight = 507.06): []
Non numeric values in column All Items (weight = 1000.00): []
Non numeric values in column Food &  Non Alcoholic Bev (weight = 518.00): []
Non numeric values in column Alcoholic Beverage, Tobacco and Kola (weight = 10.87): []
| Time                | Food   | All_items   | Food_non_alcoholic_bev   | Alcohol_tobacco_kola   |
|:--------------------|:-------|:------------|:-------------------------|:-----------------------|
| 1995-01-02 00:00:00 | 1582   | 1502        | 1563                     | 2057                   |
| 1995-01-03 00:00:00 | 1637   | 1556        | 1617                     | 216                    |
| 1995-01-04 00:00:00 | 1812   | 1695        | 1791                     | 2173                   |
| 1995-01-05 00:00:00 | 1954   | 18          | 1928                     | 2248                   |
| 1995-01-06 00:00:00 | 2068   | 1881        | 2038                     | 2367                   |


<ipython-input-3-a9f3b7785a06>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], format='%d/%m/%Y')


In [ ]:
import altair as alt

# Melt the dataframe to long format
df_melted = df.melt(id_vars='Time', var_name='CPI_Series', value_name='CPI_Value')

# Create the line chart
chart = alt.Chart(df_melted).mark_line().encode(
    x=alt.X('Time:T', axis=alt.Axis(title='Time')),
    y=alt.Y('CPI_Value:Q', axis=alt.Axis(title='CPI Value')),
    color='CPI_Series:N',
    tooltip=['Time', 'CPI_Series', 'CPI_Value']
).properties(
    title='CPI Trends Over Time'
).interactive()

# Save the chart
chart.save('cpi_trends_over_time_line_chart.json')


In [ ]:
#  plot line chart using plotly express with df_melted Time and CPI_Series

import plotly.express as px

fig = px.line(df_melted, x="Time", y="CPI_Value", color='CPI_Series', title='CPI Trends Over Time')
fig.show()


The visualization reveals some unusual jumps in the CPI values, particularly in 2014 and 2021. These are likely due to data errors or changes in the calculation methodology.

To proceed with the analysis, we will remove the data points where the CPI value is above 20000.

Given that the Food CPI is a component of both the All_items CPI and the Food_non_alcoholic_bev CPI, we will focus on analyzing the trends in these three series, as well as the Alcohol_tobacco_kola CPI.

We will analyze the trends in these series by calculating year-over-year (YoY) percentage changes.

In [ ]:
# Filter the dataframe to remove rows where any of the CPI series have values above 20000.
df_filtered = df[(df['Food'] < 20000) & (df['All_items'] < 20000) & (df['Food_non_alcoholic_bev'] < 20000) & (df['Alcohol_tobacco_kola'] < 20000)]

# Calculate the YoY percentage change for each CPI series
for col in ['Food', 'All_items', 'Food_non_alcoholic_bev', 'Alcohol_tobacco_kola']:
  df_filtered[f'{col}_yoy_pct_change'] = df_filtered[col].pct_change(periods=12) * 100

# Create a new dataframe to store the YoY percentage changes
yoy_df = df_filtered[['Time', 'Food_yoy_pct_change', 'All_items_yoy_pct_change', 'Food_non_alcoholic_bev_yoy_pct_change', 'Alcohol_tobacco_kola_yoy_pct_change']].copy()

# Print the first 5 rows of yoy_df
print(yoy_df.head().to_markdown(index=False, numalign="left", stralign="left"))


| Time                | Food_yoy_pct_change   | All_items_yoy_pct_change   | Food_non_alcoholic_bev_yoy_pct_change   | Alcohol_tobacco_kola_yoy_pct_change   |
|:--------------------|:----------------------|:---------------------------|:----------------------------------------|:--------------------------------------|
| 1995-01-02 00:00:00 | nan                   | nan                        | nan                                     | nan                                   |
| 1995-01-03 00:00:00 | nan                   | nan                        | nan                                     | nan                                   |
| 1995-01-04 00:00:00 | nan                   | nan                        | nan                                     | nan                                   |
| 1995-01-05 00:00:00 | nan                   | nan                        | nan                                     | nan                                   |
| 1995-01-06 00:00:00 | nan                   

<ipython-input-6-4b4eb72b06f6>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-4b4eb72b06f6>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-4b4eb72b06f6>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-4b4eb72b06f

The first 12 rows of the year-over-year (YoY) percentage change dataframe have null values, as there is no data from the previous year to compare with.

Next, we will visualize the YoY percentage changes to identify trends and patterns.

In [ ]:
# Drop the first 12 rows which have null values
yoy_df = yoy_df.iloc[12:].copy()

# Melt the dataframe to long format
yoy_df_melted = yoy_df.melt(id_vars='Time', var_name='CPI_Series', value_name='value')

# Create the line chart
chart = alt.Chart(yoy_df_melted).mark_line().encode(
    x=alt.X('Time:T', axis=alt.Axis(title='Time')),
    y=alt.Y('value:Q', axis=alt.Axis(title='YoY Percentage Change')),
    color='CPI_Series:N',
    tooltip=['Time', 'CPI_Series', 'value']
).properties(
    title='YoY Percentage Change in CPI Trends Over Time'
).interactive()

# Save the chart
chart.save('yoy_percentage_change_in_cpi_trends_over_time_line_chart.json')


In [ ]:
#  plot yoy_df_melted line plot Time and CPI_Series

# Plot the line chart using plotly express
fig = px.line(yoy_df_melted, x="Time", y="value", color='CPI_Series', title='YoY Percentage Change in CPI Trends Over Time')
fig.show()


The YoY percentage change plot reveals significant fluctuations and outliers, particularly in the All_items and Food_non_alcoholic_bev series. These outliers are likely due to the same data issues identified in the previous CPI value plot.

To ensure the robustness of our analysis, we will remove the extreme outliers where the YoY percentage change is above 100% or below -80%.

We will then re-visualize the YoY percentage changes and proceed with a detailed analysis of the trends.

In [ ]:
# Filter the dataframe to remove rows where any of the YoY percentage change series have values above 100% or below -80%.
yoy_df_filtered = yoy_df[
    (yoy_df['Food_yoy_pct_change'] < 100) & (yoy_df['Food_yoy_pct_change'] > -80) &
    (yoy_df['All_items_yoy_pct_change'] < 100) & (yoy_df['All_items_yoy_pct_change'] > -80) &
    (yoy_df['Food_non_alcoholic_bev_yoy_pct_change'] < 100) & (yoy_df['Food_non_alcoholic_bev_yoy_pct_change'] > -80) &
    (yoy_df['Alcohol_tobacco_kola_yoy_pct_change'] < 100) & (yoy_df['Alcohol_tobacco_kola_yoy_pct_change'] > -80)
]

# Melt the filtered dataframe to long format
yoy_df_filtered_melted = yoy_df_filtered.melt(id_vars='Time', var_name='CPI_Series', value_name='value')

# Create the line chart
chart = alt.Chart(yoy_df_filtered_melted).mark_line().encode(
    x=alt.X('Time:T', axis=alt.Axis(title='Time')),
    y=alt.Y('value:Q', axis=alt.Axis(title='YoY Percentage Change')),
    color='CPI_Series:N',
    tooltip=['Time', 'CPI_Series', 'value']
).properties(
    title='Filtered YoY Percentage Change in CPI Trends Over Time'
).interactive()

# Save the chart
chart.save('filtered_yoy_percentage_change_in_cpi_trends_over_time_line_chart.json')


In [ ]:
# plot yoy_df_melted_filtered time ad CPI_series

fig = px.line(yoy_df_filtered_melted, x="Time", y="value", color='CPI_Series', title='Filtered YoY Percentage Change in CPI Trends Over Time')
fig.show()


The filtered YoY percentage change plot shows a more stable trend, with fewer extreme outliers.

We will now analyze the average YoY percentage change for each CPI series over the entire time period, as well as for specific periods of interest:

Before 2016
After 2016

In [ ]:
# Calculate the mean YoY percentage change for each CPI series over the entire time period and print the results
print("Mean YoY percentage change for each CPI series over the entire time period:")
for col in ['Food_yoy_pct_change', 'All_items_yoy_pct_change', 'Food_non_alcoholic_bev_yoy_pct_change', 'Alcohol_tobacco_kola_yoy_pct_change']:
  print(f"{col}: {round(yoy_df_filtered[col].mean(),2)}%")

# Filter yoy_df_filtered to only include data before 2016
yoy_df_filtered_before_2016 = yoy_df_filtered[yoy_df_filtered['Time'].dt.year < 2016]

# Calculate the mean YoY percentage change for each CPI series before 2016 and print the results
print("\nMean YoY percentage change for each CPI series before 2016:")
for col in ['Food_yoy_pct_change', 'All_items_yoy_pct_change', 'Food_non_alcoholic_bev_yoy_pct_change', 'Alcohol_tobacco_kola_yoy_pct_change']:
  print(f"{col}: {round(yoy_df_filtered_before_2016[col].mean(),2)}%")

# Filter yoy_df_filtered to only include data after 2016
yoy_df_filtered_after_2016 = yoy_df_filtered[yoy_df_filtered['Time'].dt.year >= 2016]

# Calculate the mean YoY percentage change for each CPI series after 2016 and print the results
print("\nMean YoY percentage change for each CPI series after 2016:")
for col in ['Food_yoy_pct_change', 'All_items_yoy_pct_change', 'Food_non_alcoholic_bev_yoy_pct_change', 'Alcohol_tobacco_kola_yoy_pct_change']:
  print(f"{col}: {round(yoy_df_filtered_after_2016[col].mean(),2)}%")


Mean YoY percentage change for each CPI series over the entire time period:
Food_yoy_pct_change: 11.32%
All_items_yoy_pct_change: 12.19%
Food_non_alcoholic_bev_yoy_pct_change: 11.43%
Alcohol_tobacco_kola_yoy_pct_change: 11.71%

Mean YoY percentage change for each CPI series before 2016:
Food_yoy_pct_change: 11.26%
All_items_yoy_pct_change: 12.14%
Food_non_alcoholic_bev_yoy_pct_change: 11.37%
Alcohol_tobacco_kola_yoy_pct_change: 11.68%

Mean YoY percentage change for each CPI series after 2016:
Food_yoy_pct_change: 16.85%
All_items_yoy_pct_change: 16.43%
Food_non_alcoholic_bev_yoy_pct_change: 16.71%
Alcohol_tobacco_kola_yoy_pct_change: 14.44%


The analysis reveals that the average YoY percentage change for all CPI series increased after 2016 compared to the period before 2016. This suggests a general trend of increasing inflation in Nigeria, particularly in the food sector.

The Alcohol_tobacco_kola CPI series shows the lowest average YoY percentage change both before and after 2016, indicating that prices in this category have been relatively more stable compared to other categories.

In [ ]:
"""
USING LINEAR REGRESSION TO FORECAST THE FOOD_NON_ALCOHOLIC_BEV_YOY_PCT_CHANGE

"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px

# Create a new dataframe X with the column Time_trend
X = pd.DataFrame({'Time_trend': yoy_df_filtered['Time'].map(lambda x : x.timestamp())})

# Create a new dataframe y with the column Food_non_alcoholic_bev_yoy_pct_change
y = pd.DataFrame({'Food_non_alcoholic_bev_yoy_pct_change': yoy_df_filtered['Food_non_alcoholic_bev_yoy_pct_change']})

# Fit a linear regression model to predict Food_non_alcoholic_bev_yoy_pct_change using Time_trend
model = LinearRegression()
model.fit(X, y)

# Create a new dataframe future_dates with 12 rows, starting from the month after the last date in yoy_df_filtered and incrementing by one month for each subsequent row
last_date = yoy_df_filtered['Time'].max()
future_dates = pd.date_range(last_date + pd.DateOffset(months=1), periods=12, freq='MS')
future_dates_df = pd.DataFrame({'Time': future_dates})

# Add a Time_trend column to future_dates by continuing the numerical sequence from X
future_dates_df['Time_trend'] = future_dates_df['Time'].map(lambda x : x.timestamp())

# Predict the Food_non_alcoholic_bev_yoy_pct_change for the next 12 months using the linear regression model
future_predictions = model.predict(future_dates_df[['Time_trend']])

# Create a new dataframe result_df by combining future_dates and future_predictions
result_df = pd.DataFrame({'Time': future_dates, 'Food_non_alcoholic_bev_yoy_pct_change': future_predictions.flatten()})

# Print the forecasted values for the next 12 months
print("Forecasted values for the next 12 months:")
print(result_df.to_markdown(index=False, numalign="left", stralign="left"))

# Combine actual and forecasted data for plotting
plot_df = pd.concat([yoy_df_filtered[['Time', 'Food_non_alcoholic_bev_yoy_pct_change']], result_df])

# Create and display a line plot of the actual and forecasted Food_non_alcoholic_bev_yoy_pct_change values over time
fig = px.line(plot_df, x='Time', y='Food_non_alcoholic_bev_yoy_pct_change', color=plot_df['Time'] > last_date,
              labels={'Food_non_alcoholic_bev_yoy_pct_change': 'YoY % Change', 'color': 'Forecast'},
              title='Actual vs. Forecasted YoY % Change in Food & Non-Alcoholic Beverages CPI')
fig.show()


Forecasted values for the next 12 months:
| Time                | Food_non_alcoholic_bev_yoy_pct_change   |
|:--------------------|:----------------------------------------|
| 2016-03-01 00:00:00 | 10.5376                                 |
| 2016-04-01 00:00:00 | 10.5315                                 |
| 2016-05-01 00:00:00 | 10.5257                                 |
| 2016-06-01 00:00:00 | 10.5197                                 |
| 2016-07-01 00:00:00 | 10.5138                                 |
| 2016-08-01 00:00:00 | 10.5078                                 |
| 2016-09-01 00:00:00 | 10.5018                                 |
| 2016-10-01 00:00:00 | 10.4959                                 |
| 2016-11-01 00:00:00 | 10.4899                                 |
| 2016-12-01 00:00:00 | 10.4841                                 |
| 2017-01-01 00:00:00 | 10.4781                                 |
| 2017-02-01 00:00:00 | 10.472                                  |
